In [1]:
import urllib
import pandas as pd
import numpy as np
from pyspark.sql import Row
from pyspark.sql.types import StringType, FloatType, ArrayType, IntegerType
from datetime import datetime, timedelta,date
from pyspark.sql.functions import expr,first,col,when,hour,date_format,rank,udf,dense_rank
from pyspark.sql.window import Window
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.mllib.linalg import SparseVector, DenseVector
import cPickle as pickle
from pyspark.sql.functions import monotonically_increasing_id
pd.options.display.max_columns = None
pd.options.display.max_rows = None
from pyspark.mllib.recommendation import ALS,MatrixFactorizationModel,Rating
import math
sc.setCheckpointDir('checkpoint/')

print ("http://" + urllib.urlopen("http://169.254.169.254/latest/meta-data/public-hostname").read() + "/proxy/" + sc.applicationId)

http://ec2-54-74-177-140.eu-west-1.compute.amazonaws.com/proxy/application_1503834432490_0001


training data split based on odering of shows watched: user based split

In [2]:
conViewsPos = sqlContext.read.parquet("s3://c4-ati/ampapalika/rnn/dataHashed/fullDataProcessedGenre")
#a1 = conViewsPos.count()
#76,788,357

In [3]:
[f for f in conViewsPos.schema.fields]

[StructField(user,IntegerType,true),
 StructField(contract,IntegerType,true),
 StructField(episode,IntegerType,true),
 StructField(brand,StringType,true),
 StructField(conEpID,StringType,true),
 StructField(dateWatched,LongType,true),
 StructField(timeWatched,StringType,true),
 StructField(views,LongType,true),
 StructField(is_milestone_25,LongType,true),
 StructField(is_milestone_50,LongType,true),
 StructField(is_milestone_75,LongType,true),
 StructField(is_video_completion,LongType,true),
 StructField(p25,LongType,true),
 StructField(p50,LongType,true),
 StructField(p75,LongType,true),
 StructField(p100,LongType,true),
 StructField(docuVec,ArrayType(DoubleType,true),true),
 StructField(deviceDenVec,ArrayType(FloatType,true),true),
 StructField(dayPartDenVec,ArrayType(FloatType,true),true),
 StructField(dayOfWeekDenVec,ArrayType(FloatType,true),true),
 StructField(ageBinDenVec,ArrayType(FloatType,true),true),
 StructField(genderSalDenVec,ArrayType(FloatType,true),true),
 StructField(

In [ ]:
conViewsNS = sqlContext.read.parquet("s3://c4-ati/ampapalika/rnn/dataHashed/conViewsNS2T_2011-03_2016-11")

In [29]:
[f for f in conViewsNS.schema.fields]

[StructField(user,IntegerType,false),
 StructField(contract,LongType,true),
 StructField(episode,LongType,true),
 StructField(conEpID,StringType,true),
 StructField(dateWatched,LongType,true),
 StructField(timeWatched,StringType,true),
 StructField(datasource,StringType,true),
 StructField(is_milestone_25,LongType,true),
 StructField(is_milestone_50,LongType,true),
 StructField(is_milestone_75,LongType,true),
 StructField(is_video_completion,LongType,true),
 StructField(p25,LongType,true),
 StructField(p50,LongType,true),
 StructField(p75,LongType,true),
 StructField(p100,LongType,true),
 StructField(views,LongType,false)]

In [4]:
topTUsers = sqlContext.read.parquet("s3://c4-ati/ampapalika/rnn/dataHashed/top2TUsersGenre")
conViewsPos = (conViewsPos.alias("conViewsPos")
            .join(topTUsers, [topTUsers.user ==conViewsPos.user], 'inner')
            .select(conViewsPos.user,conViewsPos.conEpID,conViewsPos.timeWatched))


In [5]:
window = Window.partitionBy(conViewsPos['user']).orderBy(conViewsPos['timeWatched'])
conViewsPos = (conViewsPos.alias('conViewsPos').select('conViewsPos.*',dense_rank().over(window).alias('PUVrank')))

In [6]:
[f for f in topTUsers.schema.fields]

[StructField(user,IntegerType,true),
 StructField(count,LongType,true),
 StructField(userRank,LongType,true)]

In [7]:
topTUsers.limit(10).toPandas()

,user,count,userRank
0,354501019,492,0
1,1508131564,491,1
2,1579669784,490,2
3,192569712,480,3
4,1779100334,479,4
5,1899134431,474,5
6,1977162698,473,6
7,417008341,468,7
8,440788720,459,8
9,1161160223,458,9


In [8]:
conViewsPos.count()

481338

In [9]:
minCount = topTUsers.agg({"count": "min"}).collect()[0]["min(count)"]

In [10]:
minCount

189

In [36]:
conEpIndex = (conViewsPos.rdd
            .map(lambda x: x.conEpID)
            .distinct()
            .zipWithIndex()
            .map(lambda (conEpID,index): Row(conEpID =conEpID, conEpIndex=index))).toDF()
conEpIndex.write.parquet("s3://c4-ati/ampapalika/rnn/dataHashed/conEpIndex2TMF")

In [11]:
conEpIndex = sqlContext.read.parquet("s3://c4-ati/ampapalika/rnn/dataHashed/conEpIndex2TMF")

In [38]:
viewerIndex = (conViewsPos.rdd
            .map(lambda x: x.user)
            .distinct()
            .zipWithIndex()
            .map(lambda (viewer,index): Row(user =viewer, userIndex=index))).toDF()
viewerIndex.write.parquet("s3://c4-ati/ampapalika/rnn/dataHashed/viewerIndex2TMF")

In [12]:
viewerIndex = sqlContext.read.parquet("s3://c4-ati/ampapalika/rnn/dataHashed/viewerIndex2TMF")

In [13]:
conViewsPos = (conViewsPos.alias('conViewsPos')
               .join(viewerIndex,[viewerIndex.user==conViewsPos.user],'inner')
               .join(conEpIndex,[conEpIndex.conEpID==conViewsPos.conEpID],'inner')
               .select('conViewsPos.*',viewerIndex.userIndex,conEpIndex.conEpIndex))

In [14]:
testRank = 50

testData = (conViewsPos
                    .filter(conViewsPos.PUVrank <= testRank))

trainData = (conViewsPos
                    .filter(conViewsPos.PUVrank > testRank))

In [15]:
testData.count(), trainData.count()

(100000, 381338)

In [16]:
[f for f in testData.schema.fields]

[StructField(user,IntegerType,true),
 StructField(conEpID,StringType,true),
 StructField(timeWatched,StringType,true),
 StructField(PUVrank,IntegerType,true),
 StructField(userIndex,LongType,true),
 StructField(conEpIndex,LongType,true)]

In [17]:
testData.limit(10).toPandas()

,user,conEpID,timeWatched,PUVrank,userIndex,conEpIndex
0,291986224,61921_1,2015-10-03T20:51:24,1,1813,3579
1,291986224,59031_1,2015-10-03T21:50:16,2,1813,568
2,291986224,59031_3,2015-10-04T15:25:23,3,1813,686
3,291986224,58164_4,2015-10-04T19:48:03,4,1813,1719
4,291986224,58164_5,2015-10-04T20:39:26,5,1813,1731
5,291986224,58164_6,2015-10-04T21:48:54,6,1813,1791
6,291986224,58164_8,2015-10-04T23:39:20,7,1813,2216
7,291986224,58164_15,2015-10-05T01:37:57,8,1813,1500
8,291986224,58164_16,2015-10-05T02:29:55,9,1813,1361
9,291986224,58164_17,2015-10-05T03:20:55,10,1813,1389


Making user features that also include the last N shows that a user has watched

In [34]:
trainRatings = trainData.rdd.map(lambda y: Rating(int(y.userIndex), int(y.conEpIndex), int(1)))

In [54]:
rank = 150
iterations = 100
alpha = 40.0
lambda_=0.3

MF = ALS.trainImplicit(trainRatings, rank = rank, iterations = iterations, alpha =alpha, lambda_= lambda_)
#MF.save(sc, 's3://c4-ati/ampapalika/rnn/dataHashed/MFmodel/')

In [62]:
trainRatings.take(10)

[Rating(user=1813, product=5982, rating=1.0),
 Rating(user=1813, product=6839, rating=1.0),
 Rating(user=1813, product=6789, rating=1.0),
 Rating(user=1813, product=6266, rating=1.0),
 Rating(user=1813, product=4521, rating=1.0),
 Rating(user=1813, product=4283, rating=1.0),
 Rating(user=1813, product=4234, rating=1.0),
 Rating(user=1813, product=4184, rating=1.0),
 Rating(user=1813, product=4137, rating=1.0),
 Rating(user=1813, product=4036, rating=1.0)]

In [19]:
MF = MatrixFactorizationModel.load(sc, 's3://c4-ati/ampapalika/rnn/dataHashed/MFmodel/')


In [20]:
# Make our predictions, save output
recs_top = (MF
         .recommendProductsForUsers(15)
         .map(lambda (userIndex, items): (userIndex, [item.product for item in items]))
         .cache())

recs_top = sqlContext.createDataFrame(recs_top, ["userIndex", "conEpIndex_top_n"])


In [21]:
recs_top.limit(10).toPandas()

,userIndex,conEpIndex_top_n
0,1792,"[1390, 515, 736, 7734, 496, 7604, 987, 211, 14..."
1,1984,"[377, 448, 208, 92, 16, 2422, 3063, 127, 360, ..."
2,512,"[1390, 515, 736, 1473, 496, 987, 7734, 7604, 2..."
3,1216,"[4640, 4588, 4848, 4782, 4630, 3276, 7598, 752..."
4,704,"[377, 208, 647, 5043, 448, 281, 576, 360, 2422..."
5,320,"[3267, 1173, 1568, 3767, 1563, 3714, 1189, 321..."
6,1856,"[1139, 1450, 1298, 4025, 3949, 1497, 3979, 152..."
7,1536,"[208, 377, 92, 3063, 5751, 1855, 1812, 448, 61..."
8,128,"[377, 208, 448, 647, 281, 92, 127, 360, 576, 1..."
9,1088,"[4566, 5025, 5076, 4676, 6609, 4722, 4680, 669..."


In [67]:
recs_top.limit(10).toPandas()

,userIndex,conEpIndex_top_n
0,160,"[5751, 5692, 5551, 2308, 5967, 2028, 2193, 233..."
1,800,"[5025, 4676, 5076, 3020, 3063, 6322, 377, 4824..."
2,320,"[3063, 1855, 377, 5076, 5025, 2422, 2892, 5510..."
3,1440,"[5967, 5551, 5692, 2028, 5626, 2308, 2335, 575..."
4,480,"[6030, 1855, 5646, 1497, 7500, 682, 2989, 2308..."
5,0,"[448, 360, 2335, 5211, 4134, 3191, 1757, 682, ..."
6,1600,"[5692, 5551, 5967, 2028, 6118, 2335, 6044, 230..."
7,640,"[6609, 6915, 6820, 2670, 6952, 6950, 6603, 532..."
8,960,"[377, 3063, 2422, 448, 208, 1855, 360, 1713, 4..."
9,1120,"[6030, 5692, 3063, 5551, 1291, 2028, 5879, 575..."


In [4]:
trainDataTUNS = (conViewsNS
                 .rdd
                 .map(lambda y : Row(user_Con_eP_timeWatched_rank_device_dayPart_dayOfWeek
                                             ='{},{},{},{},{},{},{}'.format(y.userIndex,y.conEpIndex,y.timeWatched,y.fixedNSrank,
                                                                         ','.join([str(e) for e in y.deviceDenVec]),
                                                                         ','.join([str(e) for e in y.dayPartDenVec]),
                                                                         ','.join([str(e) for e in y.dayOfWeekDenVec])))
                             )).toDF()



In [ ]:
(trainDataTUNS
 .coalesce(1)
 .write
 .format("com.databricks.spark.csv")
 .option("delimiter", "|")
 .save("s3://c4-ati/ampapalika/rnn/dataHashed/MFTestResults"))

In [22]:
testRatings = testData.rdd.map(lambda y: Rating(int(y.userIndex), int(y.conEpIndex), int(1)))

In [55]:
# Evaluate the model on training data
testdata = testRatings.map(lambda p: (p[0], p[1]))
predictions = MF.predictAll(testdata).map(lambda r: ((r[0], r[1]), r[2]))
ratesAndPreds = testRatings.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)
MSE = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
print("Root Mean Squared Error = " + str(math.sqrt(MSE)))

# Save and load model
#model.save(sc, "target/tmp/myCollaborativeFilter")
#sameModel = MatrixFactorizationModel.load(sc, "target/tmp/myCollaborativeFilter")
#RMSE = 0.158344518312 on alpha= 10 training data
#RMSE = 0.0606729468497 on alpha= 40 training data
#RMSE = 0.803327080391 on alpha= 40 test data with default reg
#RMSE = 0.774325040526 on alpha= 40 test data with reg = 0.1, 10 iterations
#RMSE = 0.774325040526 on alpha= 40 test data with reg = 0.1, 100 iterations
#RMSE = 0.729284287153 on alpha= 40 test data with reg = 0.3, 100 iterations

Root Mean Squared Error = 0.729284287153


In [36]:
predictions.take(10)
#.toDF().toPandas()

[((1093, 3456), 0.022167712131611218),
 ((837, 3456), -0.01684092398889704),
 ((712, 3456), 0.04205488958383137),
 ((76, 3456), 0.06445891817001834),
 ((19, 3456), 0.06339928117786331),
 ((211, 3456), 0.038138187363151915),
 ((853, 3456), 0.011638156235528045),
 ((1366, 3456), 0.021692979921598445),
 ((471, 3456), 0.05516708062804501),
 ((1304, 3456), 0.05329843456719508)]

In [18]:
def time_period(last_day, timeWindow):
    """Return a tuple of dates, of length timeWindow, previous to the given date."""

    try:
        last_day = datetime.strptime(last_day, "%Y-%m-%d")
    except:
        pass

    set_days = [last_day - timedelta(days=x) for x in range(0, timeWindow)]
    days = tuple([day.strftime('%Y-%m-%d') for day in set_days])

    return days

last_day = '2016-11-30'
test_period=91
train_period=2000
end = datetime.strptime(last_day, '%Y-%m-%d')
date_diff = (end - timedelta(days=test_period))
train_end = date_diff.strftime("%Y-%m-%d")

test_days = time_period(end, test_period)
train_days = time_period(date_diff, train_period)
fullDuration = time_period(end, test_period+train_period)

In [22]:
fullDuration = time_period(end, test_period+train_period)
from_date = min(fullDuration)
to_date = max(fullDuration)

from_month = "-".join(from_date.split("-")[0:2])
to_month = "-".join(to_date.split("-")[0:2])
from_month,to_month

('2011-03', '2016-11')

In [23]:
fullDuration = time_period(end, test_period+train_period)
from_date = min(fullDuration)
to_date = max(fullDuration)

from_month = "-".join(from_date.split("-")[0:2])
to_month = "-".join(to_date.split("-")[0:2])

#     hit_month >= '{from_month}' AND
#     hit_month <= '{to_month}' AND
#     B.hit_month = '2016-11' AND
#     B.hit_day = '12' AND

# Find the number of views each user has for Negative samples
conViewsNS = sqlContext.sql("""SELECT
    abs(hash(scv_id)) as user,
    contract_number as contract,
    episode_number as episode,
    concat(cast(contract_number as int),'_',cast(episode_number as int)) as conEpID,
    max(local_date_key) as dateWatched,
    max(date_time) as timeWatched,
    max(datasource) as datasource,
    max(is_milestone_25) as is_milestone_25,
    max(is_milestone_50) as is_milestone_50,
    max(is_milestone_75) as is_milestone_75,
    max(is_video_completion) as is_video_completion, 
    sum(is_milestone_25) as p25,
    sum(is_milestone_50) as p50,
    sum(is_milestone_75) as p75,
    sum(is_video_completion) as p100,
    COUNT(*) as views
FROM omnitureweb_Log
WHERE
    contract_number > 0 AND 
    episode_number > 0 AND 
    is_logged_in = 1 AND
    hit_month >= '{from_month}' AND
    hit_month <= '{to_month}'
GROUP BY
    abs(hash(scv_id)),
    contract_number,
    episode_number
 """.format(from_month=from_month, to_month=to_month))
#.format(from_month=from_month, to_month=to_month)


In [24]:
topTUsers = sqlContext.read.parquet("s3://c4-ati/ampapalika/rnn/dataHashed/top2TUsersGenre")
conViewsNS = (conViewsNS.alias("conViewsNS")
            .join(topTUsers, [topTUsers.user ==conViewsNS.user], 'inner')
            .select('conViewsNS.*'))

In [25]:
conViewsNS = (conViewsNS
               .filter(conViewsNS.p25 ==0)
              .filter(conViewsNS.p50 ==0)
               .filter(conViewsNS.p75 ==0)
               .filter(conViewsNS.p100 ==0)              
              ).cache()

In [26]:
conViewsNS.limit(10).toPandas()

,user,contract,episode,conEpID,dateWatched,timeWatched,datasource,is_milestone_25,is_milestone_50,is_milestone_75,is_video_completion,p25,p50,p75,p100,views
0,349092,61204,34,61204_34,20160922,2016-09-22T02:16:02,channel4dotcom,0,0,0,0,0,0,0,0,1
1,5175312,63216,17,63216_17,20160519,2016-05-19T21:58:48,channel4dotcom,0,0,0,0,0,0,0,0,1
2,11200235,54800,1,54800_1,20150727,2015-07-27T09:35:54,channel4dotcom,0,0,0,0,0,0,0,0,1
3,11200235,60961,3,60961_3,20150926,2015-09-26T20:12:36,channel4dotcom,0,0,0,0,0,0,0,0,2
4,13718887,58207,8,58207_8,20140514,2014-05-14T14:19:15,channel4dotcom,0,0,0,0,0,0,0,0,6
5,15481881,52174,23,52174_23,20160209,2016-02-09T19:35:22,channel4dotcom,0,0,0,0,0,0,0,0,1
6,15481881,60263,1,60263_1,20161121,2016-11-21T23:02:17,channel4dotcom,0,0,0,0,0,0,0,0,1
7,19852060,60448,15,60448_15,20150525,2015-05-25T08:05:37,channel4dotcom,0,0,0,0,0,0,0,0,2
8,19852060,61526,1,61526_1,20150428,2015-04-28T17:34:47,channel4dotcom,0,0,0,0,0,0,0,0,5
9,26463312,26128,10,26128_10,20130716,2013-07-16T22:21:16,channel4dotcom,0,0,0,0,0,0,0,0,1


In [27]:
conViewsNS.count()

222832

In [28]:
conViewsNS.write.parquet("s3://c4-ati/ampapalika/rnn/dataHashed/conViewsNS2T_2011-03_2016-11")

In [ ]:
conViewsNS = sqlContext.read.parquet("s3://c4-ati/ampapalika/rnn/dataHashed/conViewsNS2T_2011-03_2016-11")

In [12]:
last_day = '2016-11-30'
test_period=91
train_period=366
end = datetime.strptime(last_day, '%Y-%m-%d')
date_diff = (end - timedelta(days=test_period))
train_end = date_diff.strftime("%Y-%m-%d")

test_days = time_period(end, test_period)
train_days = time_period(date_diff, train_period)
fullDuration = time_period(end, test_period+train_period)

fullDuration = time_period(end, test_period+train_period)
from_date = min(fullDuration)
to_date = max(fullDuration)

from_month = "-".join(from_date.split("-")[0:2])
to_month = "-".join(to_date.split("-")[0:2])
from_month, to_month

('2015-09', '2016-11')

In [13]:
conViewsNS2 = conViewsNS.filter(conViewsNS.dateWatched > 20150901)

In [14]:
conViewsNS2.count()

36972086

In [17]:
conViewsNS.count()

123667